# Abstraction Text Mining v1.2
Name: Saad Janjua and Carolyne Gakuria
Date: 9/25/2017


# Import standard packages for Machine Learning

## Imported packages and removed Deprecation Warnings.


In [1]:
#Add packages
# import module(s) into namespace
import pandas as pd   #we almost always need pandas because we like data frames
from bs4 import BeautifulSoup
import requests
import numpy as np
import nltk 
nltk.download()
from nltk import word_tokenize, pos_tag, ne_chunk
pd.set_option('display.max_colwidth', 150000) #important for getting all the text
import os
os.getcwd()
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
from sklearn.feature_extraction.text import CountVectorizer
import math
import PyPDF2 
from os.path import isfile, join
import sqlite3
import glob
import re
import os
%pwd

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


'C:\\Users\\janjuas\\Desktop'

# Change Directory to where my project is located

In [2]:
cd O:\\library\\QM\\PHI11\\Dedicated\\Quality_Improvement\\Saad\\SDS Abstraction Study\\

O:\library\QM\PHI11\Dedicated\Quality_Improvement\Saad\SDS Abstraction Study


## Read in a CSV file and Set working directories. 

In [15]:
SDS = pd.read_csv("SDS DataSet\\Tbl_2000_Member Names.csv", sep=",")
#Dimensions of dataset
print("Shape of Data", SDS.shape)
#Colum names
print("Colums Names", SDS.columns)
#See top few rows of dataset
SDS_DF = pd.DataFrame(SDS)
SDS_DF.head(2)

Shape of Data (378670, 24)
Colums Names Index(['MeasureName', 'UniversalMemberID', 'MemberID', 'ProductID', 'DOB',
       'MBR_FULL_NM', 'PROV_ID', 'PROV_NM', 'PROV_REL_GRP_PROV_ID',
       'PROV_REL_GRP_PROV_NM', 'MED_HOME_GRP_ID', 'Eligible(Admin)',
       'Num(Admin)', 'MED_HOME_LOC_ID', 'MED_HOME_LOC_DESC',
       'MBR_HOME_ADDR_LN_1', 'MBR_HOME_ADDR_LN_2', 'MBR_HOME_ADDR_CITY_NM',
       'MBR_HOME_ADDR_ST_CD', 'MBR_HOME_ADDR_ZIP_CD_5', 'MBR_HOME_ADDR_PHN_NO',
       'MaxOfMBR_FIRST_NM', 'MaxOfMBR_MIDINIT', 'MaxOfMBR_LAST_NM'],
      dtype='object')


,MeasureName,UniversalMemberID,MemberID,ProductID,DOB,MBR_FULL_NM,PROV_ID,PROV_NM,PROV_REL_GRP_PROV_ID,PROV_REL_GRP_PROV_NM,...,MED_HOME_LOC_DESC,MBR_HOME_ADDR_LN_1,MBR_HOME_ADDR_LN_2,MBR_HOME_ADDR_CITY_NM,MBR_HOME_ADDR_ST_CD,MBR_HOME_ADDR_ZIP_CD_5,MBR_HOME_ADDR_PHN_NO,MaxOfMBR_FIRST_NM,MaxOfMBR_MIDINIT,MaxOfMBR_LAST_NM
0,Adolescent Well Care Visits,10000011234,05K50237901,PCB,7/1/2002,"DEHN, AARON E",13958121,"SINGH, DALJEET",27553031,LEXINGTON MEDICAL CLINIC,...,NaN,8997 HWY FF,NaN,WELLINGTON,MO,64097,8167197498,AARON,E,DEHN
1,Adolescent Well Care Visits,10000013926,05K58888901,BCARE,2/10/1996,"HIGHTOWER BLANKS, CAMERON A",NaN,NaN,NaN,NaN,...,NaN,4327 STERLING AVENUE,NaN,KANSAS CITY,MO,64133,8167651775,CAMERON,A,HIGHTOWER BLANKS


In [4]:
"""If running locally, set these variables to your local directories."""
pdf_dir = "O:\\library\\QM\\PHI11\\Dedicated\\Quality_Improvement\\Saad\\SDS Abstraction Study\\Raw\\"
txt_dir = "O:\\library\\QM\\PHI11\\Dedicated\\Quality_Improvement\\Saad\\SDS Abstraction Study\\Processed\\"

In [5]:
"""Note: Uses a generator expression.Rerun the cell if you restart the loop below."""
corpus = (f for f in os.listdir(pdf_dir) if not f.startswith('.') and isfile(join(pdf_dir, f)))

In [6]:
"""The documentation for PyPDF2 is minimal. 
For this pattern, I followed the syntax at 
https://automatetheboringstuff.com/chapter13/ and
https://github.com/msaxton/iliff_review/blob/master/code/atla_pdfConvert.py
"""


'''This module converts PDF files from ATLASerials into plain text files. An important part of
this process is striping out the ATLA statment at the end of the PDF. In addition PyPDF tends
to read an - as Š;'''

def clean_text(string_obj):
    clean_text = re.sub("Š", "-", string_obj)
    clean_text = re.sub("¿|¡|£|^|`|«|Ł", "", clean_text)
    clean_text = re.sub("Á|Â|Ã|Ä|Å|à|á|â|ã|ä|å", "a", clean_text)
    clean_text = re.sub("È|É|Ê|Ë|è|é|ê|ë", "e", clean_text)
    clean_text = re.sub("Ì|Í|Î|Ï|ì|í|î|ï", "i", clean_text)
    clean_text = re.sub("Ò|Ó|Ô|Õ|Ö|Ø|ò|ó|ô|õ|ö|ø", "o", clean_text)
    clean_text = re.sub("Ù|Ú|Û|Ü|ù|ú|û|ü", "u", clean_text)
    clean_text = re.sub("Ý|ý|ÿ", "y", clean_text)
    return clean_text



for filename in corpus:
    # Open the PDF and load as PyPDF2 Reader object.
    pdf = open(join(pdf_dir, "CombinedFiles.pdf"),'rb')
    pdfReader = PyPDF2.PdfFileReader(pdf)
    
    
    # Loop through the pages, extract the text, and write each page to individual file.
    for page in range(0, pdfReader.numPages):
        pageObj = pdfReader.getPage(page)
        text = clean_text(pageObj.extractText())
        
        # Compile the page name. Add one because Python counts from 0.
        page_name = "{}-pg{}.txt".format(filename[:-4], page+1)

        # Write to each page to file
        with open(join(txt_dir, page_name), mode="w", encoding='utf-8') as o:
            o.write(text)

In [7]:
           
file_list = glob.glob(os.path.join(os.getcwd(), "O:\\library\\QM\\PHI11\\Dedicated\\Quality_Improvement\\Saad\\SDS Abstraction Study\\Processed\\", "*.txt"))

corpus = []

for file_path in file_list:
    with open(file_path, encoding='utf-8') as f_input:
        corpus.append(f_input.read())
        
df = pd.DataFrame.from_dict(corpus)
df.columns = ['ChartText']
df['Page'] = df.index+1
df.head(2)

,ChartText,Page
0,"8/8/2017 3:14 PM FROM: Fax Lees Summit Physicians Intemal Medicine TO: +1 (816) 995-1593 PAGE: 001 OF 018 To: User Entered Recipient Fax number: +1 (816) 995-1593 From: Johnson, Wendy Fax number: (816)525-2697 Business phone: (816)554-1918 Home phone: Date & Time: 8/8/201731445 PM Pages: 18 Re: Lees Summit Physicians Internal Medicine 1425 NW Blue Parkway LEES SUMMIT, MO 64086 SUPPORTING DOCUMENTATION FOR HEMOGLOBINA1C WENDY 816-524*3223 XT 136",1
1,"8/8/2017 3:14 PM FROM: Fax Lees Summit Physicians Intemal Medicine TO: +1 (816) 995-1593 PAGE: 002 OF 018 Lees Summit Physicians Group, Inc. 1425 NW Blue Pky. Lees Summit, MO 64086-5705 (816)524-5600 I Fax: (816)525-2697 Plltient DOB: Sex: MRN: Ordering Description 3232 Hemoglobin A1C with eAG 3230 -Hemoglobin A1C EGAN,AMYM 412611977 Femlile-F EGAAMY0002 ROBS, EMit Y G 7295· Estimated Average Glucose Order Results lab Order Clinical Order II: Report Date: Observation Date: Received Date: Out-ot-Range In-Range 5,5 111 Test performed by Laboratory Corporation of America Holdings 7800 West l10th St. Overland Park, KS 66210 AMY MEGAN DOB 04/2611977 Printed On 07/28f2017 -10:16 AM 5933099 ZZZZZOD5SZ Tue 021071201717:12:51 Tue 0210712017 Tue 021071201718:28:03 Units Expected Flag Status % mg/dl 4,0-6,0 F F F",2


In [13]:
# Connect to SQLlite database called db_Abstraction.db
conn = sqlite3.connect("db_Abstraction.db")
cur = conn.cursor()
database='db_Abstraction.db'
# Write records stored in a DataFrame to a SQL database as df_NEW and 
df.to_sql("df_NEW", conn, if_exists="replace")
SDS_DF.to_sql("SDS_DF", conn,if_exists="replace")
# Check for "?" in DataSet 
#pd.read_sql_query("select * from df_NEW;"   , conn)
pd.read_sql_query("select * from SDS_DF;"   , conn)
# Create DF for df_NEW
DF_NEW = pd.read_sql_query("select ChartText,page, CASE WHEN  ChartText  like '%Hemoglobin%' OR ChartText  like '%A1C%' OR ChartText  like '%A 1C' THEN 'yes' ELSE NULL END AS HBA1C_IN from df_NEW;", conn)
SDS_DF_NEW = pd.read_sql_query("select DOB,MBR_FULL_NM from SDS_DF WHERE MeasureName in('HbA1c Poor Control - >9','HbA1c Poor Control - No Value', 'HbA1c Testing') AND [Num(Admin)] ='Non-Compliant';", conn)

In [ ]:
SDS_DF_NEW

In [ ]:
DF_NEW

# Named Entity Recognition

In [ ]:
http://nlpforhackers.io/named-entity-extraction/

## OCR Snippet

In [148]:
#open allows you to read the file
pdfFileObj = open("CombinedFiles.pdf",'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

num_pages = pdfReader.numPages
count = 0
text = ""
#The while loop will read each page
while count < num_pages:
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()
#This if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned files.
if text != "":
   text = text
#If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text
else:
   text = textract.process(fileurl, method='tesseract', language='eng')
# Now we have a text variable which contains all the text derived #from our PDF file. Type print(text) to see what it contains. It #likely contains a lot of spaces, possibly junk such as '\n' etc.
# Now, we will clean our text variable, and return it as a list of keywords.